<a href="https://colab.research.google.com/github/Andrew-TraverseMT/placekey-joins/blob/main/CMS_hopital_and_payment_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Overlap of NPI and Overture

### Environment Setup to Read from Drive

> Using awscli, we can pull in the skinny mappings from s3



In [ ]:
!pip install polars awscli

In [ ]:
!aws s3 --no-sign-request cp s3://safegraph-public/placekey/skinny/npi/NPI_To_Placekey_Skinny_File.csv /content/data/

download: s3://safegraph-public/placekey/skinny/npi/NPI_To_Placekey_Skinny_File.csv to data/NPI_To_Placekey_Skinny_File.csv


In [ ]:
!aws s3 --no-sign-request cp s3://safegraph-public/placekey/skinny/overture/Overture_To_Placekey_Skinny_File.csv /content/data/

download: s3://safegraph-public/placekey/skinny/overture/Overture_To_Placekey_Skinny_File.csv to data/Overture_To_Placekey_Skinny_File.csv


### Diving into the Join

> At this point, you have both files to begin the join `/content/data/Overture_To_Placekey_Skinnyfile.csv` and `/content/data/NPI_To_Placekey_Skinnyfile.csv`

In [ ]:
def pct_not_null(df, column_name):
    total_non_null = df.select(pl.col(column_name).is_not_null().sum()).to_numpy()[0, 0]
    total_rows = df.height
    pct_non_null = (total_non_null / total_rows) * 100
    return round(pct_non_null, 2)

### Joining the Values
> Finally, we can complete the join. Here we are joining Overture (right) to NPI (left) to find all values of NPI that are in Overture. To promote more exploration, we compute two joins: placekey which provides location_name + address and address_placekey which joins on address.

> These are some pretty big files, so we leverage a scan of the overture data to extract only necessary values. By this, it can run a bit long - just a heads up.

In [ ]:
import polars as pl

npi = pl.read_csv('/content/data/NPI_To_Placekey_Skinny_File.csv')
npi_placekeys = npi["placekey"].to_list()
npi_address_placekeys = npi["address_placekey"].to_list()

overture = pl.scan_csv('/content/data/Overture_To_Placekey_Skinny_File.csv').filter(
            pl.col("placekey").is_in(npi_placekeys) | pl.col("address_placekey").is_in(npi_address_placekeys)
        ).collect()

In [ ]:
import polars as pl
npi_joined_overture_placekey = npi.join(overture, "placekey", "left")
npi_joined_overture_address_placekey = npi.unique("address_placekey").join(overture.unique("address_placekey"), "address_placekey", "left")
#delete npi here for memory optimzation
#delete overture here for memory optimzation
del npi
del overture
# NPI values in Overture matched on location name and address: 8.91%
print(f"NPI values in Overture matched on location name and address: " + str(pct_not_null(npi_joined_overture_placekey,"id")) + "%")
# NPI values in Overture matched on address: 24.75%
print(f"NPI values in Overture matched on address: " + str(pct_not_null(npi_joined_overture_address_placekey,"id")) + "%")

NPI values in Overture matched on location name and address: 8.77%
NPI values in Overture matched on address: 24.73%


### Pulling in the Overture Data

In [ ]:
!aws s3 --no-sign-request cp s3://overturemaps-us-west-2/release/2024-02-15-alpha.0/theme=places/type=place/ /content/data/latest --recursive

download: s3://overturemaps-us-west-2/release/2024-02-15-alpha.0/theme=places/type=place/part-00003-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet to data/latest/part-00003-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet
download: s3://overturemaps-us-west-2/release/2024-02-15-alpha.0/theme=places/type=place/part-00001-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet to data/latest/part-00001-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet
download: s3://overturemaps-us-west-2/release/2024-02-15-alpha.0/theme=places/type=place/part-00000-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet to data/latest/part-00000-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet
download: s3://overturemaps-us-west-2/release/2024-02-15-alpha.0/theme=places/type=place/part-00004-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet to data/latest/part-00004-2b783cb2-68a1-40a4-bc0c-2b129577f97b-c000.zstd.parquet
download: s3://overturemaps-us-west-2/release/2024-02-15

In [ ]:
import polars as pl
import os

ids_to_retrieve = list(set(filter(lambda item: item is not None,  (npi_joined_overture_placekey["id"].to_list() + npi_joined_overture_address_placekey["id"].to_list()))))
directory = '/content/data/latest'
dataframes = []
for filename in os.listdir(directory):
    if filename.endswith('.parquet'):
        file_path = os.path.join(directory, filename)
        df = pl.scan_parquet(file_path).filter(
            pl.col("id").is_in(ids_to_retrieve)
        ).collect()
        dataframes.append(df)
overture_df = pl.concat([df for df in dataframes])

In [ ]:
import pandas as pd
from shapely import wkb
import folium
import html

def wkb_to_lon_lat(wkb_point):
    point = wkb.loads(wkb_point)
    return point.x, point.y

def format_label(row):
    label_parts = []
    for key, value in row.items():
        if key in ['npi','names','phones','websites','socials', 'confidence', 'id']:
            sanitized_value = html.escape(str(value))
            label_parts.append(f"{key}: {sanitized_value}")
    label = ', '.join(label_parts)
    return label

def create_map(df_pre, region_filter, locality_filter):
  def filter_by_address(row, locality_filter, region_filter):
    first_address = row['addresses'][0]
    locality = first_address.get('locality')
    region = first_address.get('region')
    return (locality in locality_filter) and (region in region_filter)

  df = df_pre[df_pre.apply(filter_by_address, axis=1, locality_filter=locality_filter, region_filter=region_filter)]
  map_center = [df['latitude'].mean(), df['longitude'].mean()]
  m = folium.Map(location=map_center, zoom_start=12)

  for idx, row in df.iterrows():
      lat = float(row['latitude'])
      lon = float(row['longitude'])
      label = str(format_label(row))
      folium.Marker([lat, lon], popup=label).add_to(m)

  return m

In [ ]:
region_filter = ["NJ"]
locality_filter=["Jersey City", "Hoboken"]

In [ ]:
npi_joined_overture_placekey_with_overture = npi_joined_overture_placekey.join(overture_df, "id", "left").filter(pl.col("id").is_not_null()).to_pandas()
npi_joined_overture_placekey_with_overture[['longitude', 'latitude']] = npi_joined_overture_placekey_with_overture.apply(lambda row: wkb_to_lon_lat(row['geometry']), axis=1, result_type='expand')
del npi_joined_overture_placekey
del overture_df

In [ ]:
create_map(npi_joined_overture_placekey_with_overture, region_filter, locality_filter)